In [21]:
import sqlite3
import pandas as pd

In [61]:
# Connection à la db, crée le fichier s'il n'existe pas déjà
conn = sqlite3.connect('db_foot.sqlite3') 
c = conn.cursor()

In [62]:
# Requètes de creation des différentes tables
championships = '''CREATE TABLE IF NOT EXISTS championships (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name VARCHAR,
    country VARCHAR,
    start_year INTEGER,
    end_year INTEGER,
    created_at TIMESTAMP,
    updated_at TIMESTAMP)'''
teams = '''CREATE TABLE IF NOT EXISTS teams (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    championship_id INTEGER,
    name VARCHARD,
    city VARCHAR,
    coach_name VARCHAR,
    rank INTEGER,
    created_at TIMESTAMP,
    updated_at TIMESTAMP,
    FOREIGN KEY(championship_id) REFERENCES championships(id))'''
players = '''CREATE TABLE IF NOT EXISTS players (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    full_name VARCHAR,
    birthdate DATE,
    team_id INTEGER,
    position VARCHAR,
    nationality VARCHAR,
    created_at TIMESTAMP,
    updated_at TIMESTAMP,
    FOREIGN KEY(team_id) REFERENCES teams(id))'''
goals = '''CREATE TABLE IF NOT EXISTS goals (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    match_id INTEGER,
    player_id INTEGER,
    goal_type VARCHAR,
    created_at TIMESTAMP,
    updated_at TIMESTAMP,
    FOREIGN KEY (player_id) REFERENCES players(id),
    FOREIGN KEY (match_id) REFERENCES matchs(id))'''
matchs = '''CREATE TABLE IF NOT EXISTS matchs(
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    date DATIME,
    place VARCHAR,
    rainfall FLOAT,
    temperature FLOAT,
    created_at TIMESTAMP,
    updated_at TIMESTAMP)'''
teams_matches = '''CREATE TABLE IF NOT EXISTS teams_matches (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    match_id INTEGER,
    team_id INTEGER,
    home BOOLEAN,
    team_goals INTEGER,
    points INTEGER,
    created_at TIMESTAMP,
    updated_at TIMESTAMP,
    FOREIGN KEY (team_id) REFERENCES teams(id)
    FOREIGN KEY (match_id) REFERENCES matchs(id))'''


In [24]:
# Fonctions de création et de suppression de table
def create_table(query):
    '''Create table from query specified'''
    c.execute(query)
    conn.commit()

def drop_table(table_name):
    '''Drop table specified'''
    c.execute('''DROP TABLE %s''' % (table_name,))
    conn.commit()

In [25]:
# Fonction d'insertion 
def add_championship(name, country, start_year, end_year):
    '''Add championship in table championships with values specified'''
    c.execute('''INSERT INTO championships (name, country, start_year, end_year, created_at, updated_at) VALUES (?, ?, ?, ?, CURRENT_TIMESTAMP, CURRENT_TIMESTAMP)''', (name,country,start_year,end_year))
    conn.commit()

In [26]:
# Version pandas
def insert_table(my_df, my_table):
  '''Insert Dataframe into table specified'''
  my_df.to_sql(my_table, conn, if_exists='append', index=False)

def read_db(my_query):
  '''Read sql query, read the db an return response in a dataframe'''
  df = pd.read_sql(my_query, conn)
  return df

In [63]:
# Sandbox
# drop_table('players')
# create_table(championships)
# create_table(matchs)
# create_table(teams)
# create_table(players)
# create_table(goals)
# create_table(teams_matches)

#add_championship('Ligue 1', 'France', 2020, 2021)

In [28]:
# Lis le contenu de la table
df_championships = read_db('''SELECT * FROM championships''')
df_championships

,id,name,country,start_year,end_year,created_at,updated_at
0,1,Ligue 1,France,2020,2021,2021-06-28 12:56:07,2021-06-28 12:56:07


In [29]:
# Recup id et nom dans un dict
championships_dict = {}
for championship in range(len(df_championships)):
    championship_dict = {df_championships.iloc[championship,1] : df_championships.iloc[championship, 0]}
    championships_dict.update(championship_dict)
championships_dict

{'Ligue 1': 1}

In [38]:
df_matchs = pd.read_csv('match_table.csv')
df_matchs.drop(['Unnamed: 0'], axis=1, inplace=True)

In [39]:
df_matchs

,date,place,rainfall,temperature,created_at,updated_at
0,2020-08-21 19:00:00+02:00,Bordeaux,5.000000,24.000000,2021-06-30 12:01:37,2021-06-30 12:06:46
1,2020-08-22 17:00:00+02:00,Dijon,19.000000,26.000000,2021-06-30 12:01:37,2021-06-30 12:06:46
2,2020-08-22 21:00:00+02:00,Lille,1.000000,21.000000,2021-06-30 12:01:37,2021-06-30 12:06:46
3,2020-08-23 13:00:00+02:00,Monaco,1.000000,27.000000,2021-06-30 12:01:37,2021-06-30 12:06:46
4,2020-08-23 15:00:00+02:00,Lorient,1.000000,19.000000,2021-06-30 12:01:37,2021-06-30 12:06:46
...,...,...,...,...,...,...
375,2021-05-23 21:00:00+02:00,Rennes,2.000000,15.000000,2021-06-30 12:01:37,2021-06-30 12:06:46
376,2021-05-23 21:00:00+02:00,Nantes,1.000000,16.000000,2021-06-30 12:01:37,2021-06-30 12:06:46
377,2021-05-23 21:00:00+02:00,Saint-Étienne,4.277397,13.279778,2021-06-30 12:01:37,2021-06-30 12:06:46
378,2021-05-23 21:00:00+02:00,Reims,1.000000,14.000000,2021-06-30 12:01:37,2021-06-30 12:06:46


In [40]:
#insert_table(df_matchs, 'matchs')

read_db('''SELECT * FROM matchs''')

,id,date,place,rainfall,temperature,created_at,updated_at
0,1,2020-08-21 19:00:00+02:00,Bordeaux,5.000000,24.000000,2021-06-30 12:01:37,2021-06-30 12:06:46
1,2,2020-08-22 17:00:00+02:00,Dijon,19.000000,26.000000,2021-06-30 12:01:37,2021-06-30 12:06:46
2,3,2020-08-22 21:00:00+02:00,Lille,1.000000,21.000000,2021-06-30 12:01:37,2021-06-30 12:06:46
3,4,2020-08-23 13:00:00+02:00,Monaco,1.000000,27.000000,2021-06-30 12:01:37,2021-06-30 12:06:46
4,5,2020-08-23 15:00:00+02:00,Lorient,1.000000,19.000000,2021-06-30 12:01:37,2021-06-30 12:06:46
...,...,...,...,...,...,...,...
375,376,2021-05-23 21:00:00+02:00,Rennes,2.000000,15.000000,2021-06-30 12:01:37,2021-06-30 12:06:46
376,377,2021-05-23 21:00:00+02:00,Nantes,1.000000,16.000000,2021-06-30 12:01:37,2021-06-30 12:06:46
377,378,2021-05-23 21:00:00+02:00,Saint-Étienne,4.277397,13.279778,2021-06-30 12:01:37,2021-06-30 12:06:46
378,379,2021-05-23 21:00:00+02:00,Reims,1.000000,14.000000,2021-06-30 12:01:37,2021-06-30 12:06:46


In [46]:
df_teams = pd.read_csv('table_teams.csv', index_col=0)
df_teams

,championship_id,name,city,coach_name,rank,created_at,updated_at
0,Ligue 1,Angers Sporting Club de l'Ouest,Angers,Gérald BATICLE,44,2021-07-01 11:24:36.283421,2021-07-01 11:24:36.285113
1,Ligue 1,Football Club des Girondins de Bordeaux,Bordeaux,Jean-Louis GASSET,45,2021-07-01 11:24:36.283421,2021-07-01 11:24:36.285113
2,Ligue 1,Stade Brestois 29,Brest,Michel DER ZAKARIAN,41,2021-07-01 11:24:36.283421,2021-07-01 11:24:36.285113
3,Ligue 1,Dijon Football Côte-d'Or,Dijon,David LINARES,21,2021-07-01 11:24:36.283421,2021-07-01 11:24:36.285113
4,Ligue 1,Racing Club de Lens,Lens,Franck HAISE,57,2021-07-01 11:24:36.283421,2021-07-01 11:24:36.285113
5,Ligue 1,LOSC Lille,Lille,Christophe GALTIER,83,2021-07-01 11:24:36.283421,2021-07-01 11:24:36.285113
6,Ligue 1,Football Club Lorient-Bretagne Sud,Lorient,Christophe PELISSIER,42,2021-07-01 11:24:36.283421,2021-07-01 11:24:36.285113
7,Ligue 1,Olympique Lyonnais,Lyon,Peter BOSZ,76,2021-07-01 11:24:36.283421,2021-07-01 11:24:36.285113
8,Ligue 1,Olympique de Marseille,Marseille,Jorge SAMPAOLI,60,2021-07-01 11:24:36.283421,2021-07-01 11:24:36.285113
9,Ligue 1,Football Club de Metz,Metz,Frédéric ANTONETTI,47,2021-07-01 11:24:36.283421,2021-07-01 11:24:36.285113


In [47]:
#map colonne avec dictionnaire des ids
df_teams['championship_id'] = df_teams['championship_id'].map(championships_dict)
df_teams

,championship_id,name,city,coach_name,rank,created_at,updated_at
0,1,Angers Sporting Club de l'Ouest,Angers,Gérald BATICLE,44,2021-07-01 11:24:36.283421,2021-07-01 11:24:36.285113
1,1,Football Club des Girondins de Bordeaux,Bordeaux,Jean-Louis GASSET,45,2021-07-01 11:24:36.283421,2021-07-01 11:24:36.285113
2,1,Stade Brestois 29,Brest,Michel DER ZAKARIAN,41,2021-07-01 11:24:36.283421,2021-07-01 11:24:36.285113
3,1,Dijon Football Côte-d'Or,Dijon,David LINARES,21,2021-07-01 11:24:36.283421,2021-07-01 11:24:36.285113
4,1,Racing Club de Lens,Lens,Franck HAISE,57,2021-07-01 11:24:36.283421,2021-07-01 11:24:36.285113
5,1,LOSC Lille,Lille,Christophe GALTIER,83,2021-07-01 11:24:36.283421,2021-07-01 11:24:36.285113
6,1,Football Club Lorient-Bretagne Sud,Lorient,Christophe PELISSIER,42,2021-07-01 11:24:36.283421,2021-07-01 11:24:36.285113
7,1,Olympique Lyonnais,Lyon,Peter BOSZ,76,2021-07-01 11:24:36.283421,2021-07-01 11:24:36.285113
8,1,Olympique de Marseille,Marseille,Jorge SAMPAOLI,60,2021-07-01 11:24:36.283421,2021-07-01 11:24:36.285113
9,1,Football Club de Metz,Metz,Frédéric ANTONETTI,47,2021-07-01 11:24:36.283421,2021-07-01 11:24:36.285113


In [52]:
#insert_table(df_teams, 'teams')

df_teams_table = read_db('''SELECT * FROM teams''')
df_teams_table

,id,championship_id,name,city,coach_name,rank,created_at,updated_at
0,1,1,Angers Sporting Club de l'Ouest,Angers,Gérald BATICLE,44,2021-07-01 11:24:36.283421,2021-07-01 11:24:36.285113
1,2,1,Football Club des Girondins de Bordeaux,Bordeaux,Jean-Louis GASSET,45,2021-07-01 11:24:36.283421,2021-07-01 11:24:36.285113
2,3,1,Stade Brestois 29,Brest,Michel DER ZAKARIAN,41,2021-07-01 11:24:36.283421,2021-07-01 11:24:36.285113
3,4,1,Dijon Football Côte-d'Or,Dijon,David LINARES,21,2021-07-01 11:24:36.283421,2021-07-01 11:24:36.285113
4,5,1,Racing Club de Lens,Lens,Franck HAISE,57,2021-07-01 11:24:36.283421,2021-07-01 11:24:36.285113
5,6,1,LOSC Lille,Lille,Christophe GALTIER,83,2021-07-01 11:24:36.283421,2021-07-01 11:24:36.285113
6,7,1,Football Club Lorient-Bretagne Sud,Lorient,Christophe PELISSIER,42,2021-07-01 11:24:36.283421,2021-07-01 11:24:36.285113
7,8,1,Olympique Lyonnais,Lyon,Peter BOSZ,76,2021-07-01 11:24:36.283421,2021-07-01 11:24:36.285113
8,9,1,Olympique de Marseille,Marseille,Jorge SAMPAOLI,60,2021-07-01 11:24:36.283421,2021-07-01 11:24:36.285113
9,10,1,Football Club de Metz,Metz,Frédéric ANTONETTI,47,2021-07-01 11:24:36.283421,2021-07-01 11:24:36.285113


In [54]:
# Recup id et nom dans un dict
teams_dict = {}
for team in range(len(df_teams_table)):
    team_dict = {
        df_teams_table.iloc[team, 2]: df_teams_table.iloc[team, 0]}
    teams_dict.update(team_dict)
teams_dict


{"Angers Sporting Club de l'Ouest": 1,
 'Football Club des Girondins de Bordeaux': 2,
 'Stade Brestois 29': 3,
 "Dijon Football Côte-d'Or": 4,
 'Racing Club de Lens': 5,
 'LOSC Lille': 6,
 'Football Club Lorient-Bretagne Sud': 7,
 'Olympique Lyonnais': 8,
 'Olympique de Marseille': 9,
 'Football Club de Metz': 10,
 'Association sportive de Monaco FC': 11,
 'Montpellier-Herault Sport Club': 12,
 'Football Club de Nantes': 13,
 "Olympique Gymnaste Club Nice Côte d'Azur": 14,
 'Nîmes Olympique': 15,
 'Paris-Saint-Germain Football Club': 16,
 'Stade de Reims': 17,
 'Stade Rennais Football Club': 18,
 'Association Sportive de St-Etienne': 19,
 'Racing Club de Strasbourg Alsace': 20}

In [56]:
df_players = pd.read_csv('table_players.csv', index_col=0)
df_players

,full_name,birthdate,team_id,position,nationality,created_at,updated_at
0,P. Bernardoni,18/04/97,Angers Sporting Club de l'Ouest,Gar.,FRA,2021-07-01 13:56:41.648858,2021-07-01 13:56:41.652471
1,L. Butelle,3/04/83,Angers Sporting Club de l'Ouest,Gar.,FRA,2021-07-01 13:56:41.648858,2021-07-01 13:56:41.652471
2,D. Petkovic,25/05/93,Angers Sporting Club de l'Ouest,Gar.,MNE,2021-07-01 13:56:41.648858,2021-07-01 13:56:41.652471
3,A. Bamba,25/04/90,Angers Sporting Club de l'Ouest,Déf.,CIV,2021-07-01 13:56:41.648858,2021-07-01 13:56:41.652471
4,K. Boma,20/11/02,Angers Sporting Club de l'Ouest,Déf.,FRA,2021-07-01 13:56:41.648858,2021-07-01 13:56:41.652471
...,...,...,...,...,...,...,...
631,I. Saadi,8/02/92,Racing Club de Strasbourg Alsace,Att.,ALG,2021-07-01 13:56:41.648858,2021-07-01 13:56:41.652471
632,M. Sahi,20/12/01,Racing Club de Strasbourg Alsace,Att.,MLI,2021-07-01 13:56:41.648858,2021-07-01 13:56:41.652471
633,A. Waris,19/09/91,Racing Club de Strasbourg Alsace,Att.,GHA,2021-07-01 13:56:41.648858,2021-07-01 13:56:41.652471
634,K. Zohi,19/12/96,Racing Club de Strasbourg Alsace,Att.,CIV,2021-07-01 13:56:41.648858,2021-07-01 13:56:41.652471


In [57]:
df_players['team_id'] = df_players['team_id'].map(teams_dict)
df_players

,full_name,birthdate,team_id,position,nationality,created_at,updated_at
0,P. Bernardoni,18/04/97,1,Gar.,FRA,2021-07-01 13:56:41.648858,2021-07-01 13:56:41.652471
1,L. Butelle,3/04/83,1,Gar.,FRA,2021-07-01 13:56:41.648858,2021-07-01 13:56:41.652471
2,D. Petkovic,25/05/93,1,Gar.,MNE,2021-07-01 13:56:41.648858,2021-07-01 13:56:41.652471
3,A. Bamba,25/04/90,1,Déf.,CIV,2021-07-01 13:56:41.648858,2021-07-01 13:56:41.652471
4,K. Boma,20/11/02,1,Déf.,FRA,2021-07-01 13:56:41.648858,2021-07-01 13:56:41.652471
...,...,...,...,...,...,...,...
631,I. Saadi,8/02/92,20,Att.,ALG,2021-07-01 13:56:41.648858,2021-07-01 13:56:41.652471
632,M. Sahi,20/12/01,20,Att.,MLI,2021-07-01 13:56:41.648858,2021-07-01 13:56:41.652471
633,A. Waris,19/09/91,20,Att.,GHA,2021-07-01 13:56:41.648858,2021-07-01 13:56:41.652471
634,K. Zohi,19/12/96,20,Att.,CIV,2021-07-01 13:56:41.648858,2021-07-01 13:56:41.652471


In [64]:
#insert_table(df_players, 'players')

df_players_table = read_db('''SELECT * FROM players''')
df_players_table


,id,full_name,birthdate,team_id,position,nationality,created_at,updated_at
0,1,P. Bernardoni,18/04/97,1,Gar.,FRA,2021-07-01 13:56:41.648858,2021-07-01 13:56:41.652471
1,2,L. Butelle,3/04/83,1,Gar.,FRA,2021-07-01 13:56:41.648858,2021-07-01 13:56:41.652471
2,3,D. Petkovic,25/05/93,1,Gar.,MNE,2021-07-01 13:56:41.648858,2021-07-01 13:56:41.652471
3,4,A. Bamba,25/04/90,1,Déf.,CIV,2021-07-01 13:56:41.648858,2021-07-01 13:56:41.652471
4,5,K. Boma,20/11/02,1,Déf.,FRA,2021-07-01 13:56:41.648858,2021-07-01 13:56:41.652471
...,...,...,...,...,...,...,...,...
631,632,I. Saadi,8/02/92,20,Att.,ALG,2021-07-01 13:56:41.648858,2021-07-01 13:56:41.652471
632,633,M. Sahi,20/12/01,20,Att.,MLI,2021-07-01 13:56:41.648858,2021-07-01 13:56:41.652471
633,634,A. Waris,19/09/91,20,Att.,GHA,2021-07-01 13:56:41.648858,2021-07-01 13:56:41.652471
634,635,K. Zohi,19/12/96,20,Att.,CIV,2021-07-01 13:56:41.648858,2021-07-01 13:56:41.652471
